# Symmetry analysis 
The objective of this study is to investigate the feasibility of character expression recognition in the context of face text.

In [12]:
import os, hydra, warnings

warnings.filterwarnings("ignore")

import logging, time, sys, json, yaml, csv, shutil, copy

sys.path.append("/workspace/skeleton/")
sys.path.append("/workspace/skeleton/project")
from pathlib import Path
import torch

from torchmetrics.classification import (
    MulticlassAccuracy,
    MulticlassPrecision,
    MulticlassRecall,
    MulticlassF1Score,
    MulticlassConfusionMatrix,
    MulticlassAUROC,
)


from project.dataloader.data_loader import WalkDataModule
from project.trainer.train_temporal_mix import TemporalMixModule

from pytorch_lightning import seed_everything

from pytorch_grad_cam import (
    GradCAM,
    ScoreCAM,
    GradCAMPlusPlus,
    AblationCAM,
    XGradCAM,
    EigenCAM,
    EigenGradCAM,
    LayerCAM,
)
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.io.image import write_png



In [13]:
ckpt = '/workspace/skeleton/logs/paper/temporal_mix/2/version_0/checkpoints/6-0.57-0.6770.ckpt'
hparams = torch.load(ckpt, map_location='cpu')['hyper_parameters']['hparams']
fold = ckpt.split("/")[-4]


In [14]:
# 因为代码进行过修改，之前的配置文件不再适用，需要重新设置/增加一些参数
from omegaconf import OmegaConf, open_dict
OmegaConf.set_struct(hparams, True)
with open_dict(hparams):
	hparams.train.experiment = 'temporal_mix'
	hparams.train.transfer_learning = True
	hparams.data.default_batch_size = 1
hparams.train.experiment

'temporal_mix'

In [15]:
val_dataset_idx = "/workspace/data/seg_gait_index_mapping/2/over/index.json"
# prepare dataset mapping     

fold_dataset_idx = json.load(open(val_dataset_idx, "r"))

# unpack the dataset mapping
for k, v in fold_dataset_idx.items():
	# train mapping, include the gait cycle index
	train_mapping_idx = v[0]
	fold_dataset_idx[k][0] = [Path(i) for i in train_mapping_idx]

	# val mapping, include the gait cycle index 
	val_mapping_idx = v[1]
	fold_dataset_idx[k][1] = [Path(i) for i in val_mapping_idx]

	# train video path
	train_video_idx = v[2]
	fold_dataset_idx[k][2] = Path(train_video_idx)

	# val video path
	val_dataset_idx = v[3]
	fold_dataset_idx[k][3] = Path(val_dataset_idx)

seed_everything(42, workers=True)

Global seed set to 42


42

In [16]:
hparams.train.transfer_learning

True

In [17]:
model = TemporalMixModule.load_from_checkpoint(checkpoint_path=ckpt, hparams=hparams)
data_module = WalkDataModule(hparams, fold_dataset_idx[fold])
data_module.setup()

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [18]:
# inference one sample
for i, batch in enumerate(data_module.test_dataloader()):
    
	video = batch["video"].detach().cpu()
	label = batch["label"].detach().cpu()

	model.eval()

	if i == 0: break

video name: 20180521_2_ASD_lat__V1-0004, gait cycle index: [0, 11, 26, 38, 53, 66, 81, 95, 108, 121, 133, 138]
used split gait cycle index: [0, 26, 53, 81, 108, 133]
used split gait cycle index: [11, 38, 66, 95, 121]
video name: 20191029_2_LCS_lat_V1-0001, gait cycle index: [0, 11, 26, 43, 57, 74, 90, 106, 123, 147]
used split gait cycle index: [0, 26, 57, 90, 123]
used split gait cycle index: [11, 43, 74, 106]
video name: 20180723_2_ASD_lat__V1-0002, gait cycle index: [0, 28, 39, 64, 94, 105, 116, 132, 165, 176, 187, 203, 214, 235, 246, 270, 281, 303, 314, 338, 349, 371, 382, 401, 412, 430, 441, 450]
used split gait cycle index: [0, 39, 94, 116, 165, 187, 214, 246, 281, 314, 349, 382, 412, 441]
used split gait cycle index: [28, 64, 105, 132, 176, 203, 235, 270, 303, 338, 371, 401, 430]
video name: 20180723_2_ASD_lat__V1-0014, gait cycle index: [0, 15, 26, 49, 60, 80, 91, 115, 126, 151, 185, 196, 219, 230, 249, 260, 282, 293, 314, 325, 349, 360, 385, 396, 417, 428, 446, 457, 483, 488]


In [19]:
with torch.no_grad():
    preds = model(video)

In [26]:
preds[0], label[0]

(tensor([-0.6522, -3.1424,  3.0430]), tensor(2))

In [29]:
batch['info'][-1]

{'video': tensor([[[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            ...,
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
           [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            ...,
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
           [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],

In [21]:
torch.softmax(preds, dim=1), label

(tensor([[0.0242, 0.0020, 0.9738],
         [0.0130, 0.0020, 0.9850],
         [0.0109, 0.0023, 0.9868],
         [0.0259, 0.0020, 0.9721],
         [0.0499, 0.0035, 0.9466],
         [0.9619, 0.0136, 0.0245],
         [0.9815, 0.0075, 0.0110],
         [0.9698, 0.0122, 0.0180],
         [0.9755, 0.0103, 0.0143],
         [0.8824, 0.0408, 0.0768],
         [0.9650, 0.0113, 0.0238],
         [0.9741, 0.0102, 0.0157],
         [0.9778, 0.0094, 0.0129],
         [0.9686, 0.0134, 0.0180],
         [0.9465, 0.0151, 0.0384],
         [0.3968, 0.5509, 0.0523],
         [0.3355, 0.5732, 0.0913],
         [0.2494, 0.6615, 0.0890],
         [0.2877, 0.6215, 0.0908],
         [0.3143, 0.5915, 0.0941],
         [0.3807, 0.5241, 0.0952],
         [0.3304, 0.4033, 0.2663],
         [0.1472, 0.7081, 0.1447],
         [0.1097, 0.7562, 0.1341],
         [0.3221, 0.3119, 0.3660],
         [0.3864, 0.1775, 0.4361],
         [0.3168, 0.2146, 0.4686],
         [0.2234, 0.2000, 0.5767],
         [0.2621, 0.

# Darw Symmetry Line

In [22]:
fold_dataset_idx[fold]

[[PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160120_ASD_lat__V1-0001.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160120_ASD_lat__V1-0002.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160120_ASD_lat__V1-0003.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160120_ASD_lat__V1-0004.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160120_ASD_lat__V1-0005.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160120_ASD_lat__V1-0006.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160120_ASD_lat__V1-0007.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160426_ASD_lat__V1-0001.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160426_ASD_lat__V1-0002.json'),
  PosixPath('/workspace/data/segmentation_dataset_512/json_mix/ASD/20160426_ASD_lat__V1-000